In [ ]:
# Install Keras Tuner
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.7 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
# Data collection and transformation Packages
import pandas as pd
import numpy as np

# Tensorflow and keras tuning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical
import keras_tuner as kt

# sklearn for dataset and train, test, split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# small visualization at end
import seaborn as sns

In [ ]:
df = pd.read_csv("diabetes.csv")

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.corr()["Outcome"]

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [ ]:
#Create our train test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                          test_size=0.2, random_state=42)

## Building our first model

In [ ]:
#Create sequential model
model = keras.Sequential()

# Add an input layer, build it any which way
model.add(layers.Input(shape=(8,)))
model.add(layers.Dense(32, activation='relu'))

# Add an output layer
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Use earlystopping to prevent overfitting once validation loss
# change is minimal
early_stop = EarlyStopping(monitor='val_loss', patience=10, start_from_epoch=70)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32,
         validation_data=(X_test, y_test), callbacks=[early_stop],
          verbose=1)


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3297 - loss: 0.8459 - val_accuracy: 0.3636 - val_loss: 0.7883
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3667 - loss: 0.7704 - val_accuracy: 0.4740 - val_loss: 0.7201
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5193 - loss: 0.7000 - val_accuracy: 0.6169 - val_loss: 0.6712
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6785 - loss: 0.6421 - val_accuracy: 0.6753 - val_loss: 0.6334
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7544 - loss: 0.5878 - val_accuracy: 0.6883 - val_loss: 0.6073
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7384 - loss: 0.5681 - val_accuracy: 0.7013 - val_loss: 0.5865
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7541 - loss: 0.5495 - val_accuracy: 0.7208 - val_loss: 0.5708
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7722 - loss: 0.5230 - val_accuracy: 0.7273 - 

In [ ]:
#Using argmax to get the index of the highest value
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).reshape(-1)
y_pred[:3], y_test[:3]

1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


(array([0, 0, 0]), array([0, 0, 0]))

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.79        99
           1       0.63      0.62      0.62        55

    accuracy                           0.73       154
   macro avg       0.71      0.71      0.71       154
weighted avg       0.73      0.73      0.73       154



## Building out and using the tuner

In [ ]:
# Start by creating the function
def model_builder(hp):
    # Let it know what model we want, in this case, sequential
    model = keras.Sequential()

    #This is our initial input layer.
    #This will generally match the data we are using
    model.add(layers.Input(shape=(8,)))
    #This is the magical chunk here
    #We are not sure how many layers is ideal
    #So we tell the tuner, hey, try anywhere from 1-3 layers
    for i in range(hp.Int('num_dense_layers', 1, 8)):
        model.add(layers.Dense(units=hp.Int('dense_' + str(i),
                     min_value=8, max_value=128, step=8),
                  activation=hp.Choice('activation'+str(i), values = ['relu', 'selu', 'elu', 'silu'])))
        model.add(layers.Dropout(hp.Choice('dropout_'+ str(i),
                      values=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6])))
    #This is our output layer, just like normal
    #There are 3 potential outcomes so we set it as 3 nodes
    model.add(layers.Dense(1, activation='sigmoid'))
    # Decide between one of two optimizers
    # If 'Adam' or 'SGD' test the appropriate parameters
    hp_optimizer=hp.Choice('Optimizer', values=['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Nadam'])
    if hp_optimizer == 'Adam':
        hp_learning_rate = hp.Choice('learning_rate',
                            values=[1e-1, 1e-2, 1e-3])
    elif hp_optimizer == 'SGD':
        hp_learning_rate = hp.Choice('learning_rate',
                                      values=[1e-1, 1e-2, 1e-3])
        nesterov=True
        momentum=0.9

    #Compile and return the model
    model.compile(optimizer = hp_optimizer, loss='binary_crossentropy',
                    metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.tuners.BayesianOptimization(
        model_builder, #The model function we wrote
        seed=777, #Seed for evaluation
        objective='val_loss', #Which objective to keep track of
        max_trials=30, #Number of trials to try
        directory='.', #Directory to save tuning files to
        overwrite=True, #Whether to overwrite each run
        project_name='tuning-diabetes') #Name of the tuning files

# tuner.search_space_summary # will tell you the search space

In [ ]:
#This will perform the actual search
#Search on our training data, validate with test data
#Use early callback to prevent overfitting
# Set epoch min because you want to get the best parameters not train a model
# After getting parameters you can train model on those values
tuner.search(X_train, y_train, epochs=10,
              validation_data=(X_test, y_test),
              callbacks=[early_stop])

Trial 30 Complete [00h 00m 04s]
val_loss: 0.7079535126686096

Best val_loss So Far: 0.4770575165748596
Total elapsed time: 00h 02m 45s


In [ ]:
# Get the top 2 models.
tuner.get_best_hyperparameters()[0].values


{'num_dense_layers': 2,
 'dense_0': 80,
 'activation0': 'elu',
 'dropout_0': 0.5,
 'Optimizer': 'Adam',
 'learning_rate': 0.01,
 'dense_1': 128,
 'activation1': 'selu',
 'dropout_1': 0.0,
 'dense_2': 24,
 'activation2': 'silu',
 'dropout_2': 0.4,
 'dense_3': 104,
 'activation3': 'elu',
 'dropout_3': 0.5,
 'dense_4': 8,
 'activation4': 'elu',
 'dropout_4': 0.3,
 'dense_5': 56,
 'activation5': 'relu',
 'dropout_5': 0.3,
 'dense_6': 104,
 'activation6': 'selu',
 'dropout_6': 0.4,
 'dense_7': 80,
 'activation7': 'selu',
 'dropout_7': 0.2}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │             720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 80)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          10,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,217 (43.82 KB)

 Trainable params: 11,217 (43.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# initial_epoch: Integer. Epoch at which to start training (useful for resuming a previous training run)
model.fit(X_train, y_train, batch_size=32, epochs=200, initial_epoch=10, validation_data=(X_test, y_test))

Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7837 - loss: 0.4772 - val_accuracy: 0.7338 - val_loss: 0.5224
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7650 - loss: 0.4787 - val_accuracy: 0.7532 - val_loss: 0.4923
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7515 - loss: 0.4887 - val_accuracy: 0.7662 - val_loss: 0.5155
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7878 - loss: 0.4520 - val_accuracy: 0.7273 - val_loss: 0.5236
Epoch 15/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7807 - loss: 0.5072 - val_accuracy: 0.7662 - val_loss: 0.4922
Epoch 16/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7898 - loss: 0.4629 - val_accuracy: 0.7338 - val_loss: 0.5069
Epoch 17/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7662 - loss: 0.4576 - val_accuracy: 0.7532 - val_loss: 0.5021
Epoch 18/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7880 - loss: 0.4784 - val_accuracy: 0

In [99]:
#Using argmax to get the index of the highest value
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).reshape(-1)
y_pred[:3], y_test[:3]

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


(array([1, 0, 0]), array([0, 0, 0]))

In [100]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.82      0.82        99
           1       0.68      0.69      0.68        55

    accuracy                           0.77       154
   macro avg       0.75      0.75      0.75       154
weighted avg       0.77      0.77      0.77       154

